In [6]:
import numpy as np
import torch
from datasets import load_dataset, Value, Sequence
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
    DebertaV2Tokenizer
)
import pandas as pd
from tabulate import tabulate
from sklearn.metrics import f1_score

# ------------------------------
# CUDA Initialization Check
# ------------------------------
if torch.cuda.is_available():
    try:
        torch.cuda.init()  # Force CUDA initialization (optional)
    except Exception as e:
        print("Error during CUDA initialization:", e)
    print("CUDA is available. Running on GPU.")
    try:
        print("GPU Name:", torch.cuda.get_device_name(0))
    except Exception as e:
        print("Could not get GPU Name:", e)
    print("GPU Count:", torch.cuda.device_count())
else:
    print("CUDA is not available. Running on CPU.")

# ------------------------------
# 1. Load the GoEmotions Dataset
# ------------------------------
dataset = load_dataset("go_emotions")
print("\nSample from training set:")
print(dataset["train"][0])  # Show a sample to inspect

# Number of labels
num_labels = dataset["train"].features["labels"].feature.num_classes
print("Number of labels:", num_labels)

# ------------------------------
# 2. Data Preprocessing
# ------------------------------
def convert_labels(label_list):
    """Convert list of label indices to a multi-hot vector of floats."""
    return [1.0 if i in label_list else 0.0 for i in range(num_labels)]

model_checkpoint = "microsoft/deberta-v3-base"
# Use a stronger checkpoint such as microsoft/deberta-v3-base
tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base")
max_length = 128  # You can experiment with longer sequences

def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    tokenized["labels"] = [convert_labels(lbls) for lbls in examples["labels"]]
    return tokenized

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.cast_column("labels", Sequence(Value("float32")))
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ------------------------------
# 3. Model Setup
# ------------------------------
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
)
model.config.problem_type = "multi_label_classification"

if torch.cuda.is_available():
    model = model.to("cuda")

# ------------------------------
# 4. Define Compute Metrics Function
# ------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Binarize predictions using threshold 0.5
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    # Compute subset accuracy (exact match)
    subset_accuracy = np.mean(np.all(predictions == labels, axis=1))
    # Compute micro and macro F1
    micro_f1 = f1_score(labels, predictions, average="micro", zero_division=0)
    macro_f1 = f1_score(labels, predictions, average="macro", zero_division=0)
    return {
        "subset_accuracy": subset_accuracy,
        "micro_f1": micro_f1,
        "macro_f1": macro_f1
    }

# ------------------------------
# 5. Training Arguments & Trainer Setup
# ------------------------------
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # Try 5-10 epochs for better performance
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
)

def custom_data_collator(features):
    batch = DefaultDataCollator()(features)
    batch["labels"] = batch["labels"].float()
    return batch

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=custom_data_collator,
    compute_metrics=compute_metrics,
)

print("\nStarting training...")
trainer.train()

# ------------------------------
# 6. Evaluation
# ------------------------------
print("\nEvaluating on validation set...")
val_results = trainer.evaluate()
df_val_results = pd.DataFrame([val_results])
print("Validation Results:")
print(tabulate(df_val_results, headers="keys", tablefmt="fancy_grid"))

print("\nEvaluating on test set...")
test_results = trainer.evaluate(eval_dataset=encoded_dataset["test"])
df_test_results = pd.DataFrame([test_results])
print("Test Results:")
print(tabulate(df_test_results, headers="keys", tablefmt="fancy_grid"))

# ------------------------------
# 7. Inference Example
# ------------------------------
def predict_emotions(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits[0]
    probs = torch.sigmoid(logits).cpu().numpy()
    return probs

sample_text = "I am feeling really happy and excited about today!"
probs = predict_emotions(sample_text)
print("\nSample inference text:", sample_text)
print("Predicted probabilities for each emotion:")
print(probs)


CUDA is available. Running on GPU.
GPU Name: NVIDIA GeForce RTX 4070 Ti SUPER
GPU Count: 1

Sample from training set:
{'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'id': 'eebbqej'}
Number of labels: 28


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Second_Sem\csc392\env_py310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saadu\.cache\huggingface\hub\models--microsoft--deberta-v3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(mes


Starting training...


  1%|          | 101/13570 [00:13<27:54,  8.04it/s]

{'loss': 0.3424, 'learning_rate': 1.9852616064848935e-05, 'epoch': 0.04}


...


Evaluating on validation set...


100%|██████████| 340/340 [00:11<00:00, 29.56it/s]


Validation Results:
╒════╤═════════════╤════════════════════════╤═════════════════╤═════════════════╤════════════════╤═══════════════════════════╤═════════════════════════╤═════════╕
│    │   eval_loss │   eval_subset_accuracy │   eval_micro_f1 │   eval_macro_f1 │   eval_runtime │   eval_samples_per_second │   eval_steps_per_second │   epoch │
╞════╪═════════════╪════════════════════════╪═════════════════╪═════════════════╪════════════════╪═══════════════════════════╪═════════════════════════╪═════════╡
│  0 │   0.0844298 │               0.460192 │         0.57608 │        0.441173 │        11.5908 │                   468.128 │                  29.333 │       5 │
╘════╧═════════════╧════════════════════════╧═════════════════╧═════════════════╧════════════════╧═══════════════════════════╧═════════════════════════╧═════════╛

Evaluating on test set...


100%|██████████| 340/340 [00:11<00:00, 29.77it/s]

Test Results:
╒════╤═════════════╤════════════════════════╤═════════════════╤═════════════════╤════════════════╤═══════════════════════════╤═════════════════════════╤═════════╕
│    │   eval_loss │   eval_subset_accuracy │   eval_micro_f1 │   eval_macro_f1 │   eval_runtime │   eval_samples_per_second │   eval_steps_per_second │   epoch │
╞════╪═════════════╪════════════════════════╪═════════════════╪═════════════════╪════════════════╪═══════════════════════════╪═════════════════════════╪═════════╡
│  0 │   0.0840901 │                0.46066 │        0.576743 │        0.441563 │        11.4569 │                   473.688 │                  29.676 │       5 │
╘════╧═════════════╧════════════════════════╧═════════════════╧═════════════════╧════════════════╧═══════════════════════════╧═════════════════════════╧═════════╛

Sample inference text: I am feeling really happy and excited about today!
Predicted probabilities for each emotion:
[5.21914177e-02 1.28694540e-02 8.52493919e-04 1.108821

In [5]:
# Use a specific prompt for inference
prompt_text = "I want a burger"
probs = predict_emotions(prompt_text)

# Get the label names from the dataset (GoEmotions)
label_names = dataset["train"].features["labels"].feature.names

threshold = 0.5 

# Convert probabilities to binary predictions
predicted_binary = (probs > threshold).astype(int)

# For multi-label classification, there may be multiple labels predicted.
predicted_labels = [label_names[i] for i, pred in enumerate(predicted_binary) if pred == 1]

print("\nPredicted labels (threshold={}):".format(threshold))
print(predicted_labels)



Predicted labels (threshold=0.5):
['desire']


In [3]:
import transformers
print(transformers.__version__)

4.31.0


In [7]:
trainer.save_model("./my_saved_model")
tokenizer.save_pretrained("./my_saved_model")

('./my_saved_model\\tokenizer_config.json',
 './my_saved_model\\special_tokens_map.json',
 './my_saved_model\\spm.model',
 './my_saved_model\\added_tokens.json')

In [8]:
import numpy as np
import torch
from datasets import load_dataset, Value, Sequence
from transformers import (
    DebertaV2Tokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)
import pandas as pd
from tabulate import tabulate
from sklearn.metrics import f1_score

# ------------------------------
# CUDA Initialization Check
# ------------------------------
if torch.cuda.is_available():
    try:
        torch.cuda.init()  # Force CUDA initialization (optional)
    except Exception as e:
        print("Error during CUDA initialization:", e)
    print("CUDA is available. Running on GPU.")
    try:
        print("GPU Name:", torch.cuda.get_device_name(0))
    except Exception as e:
        print("Could not get GPU Name:", e)
    print("GPU Count:", torch.cuda.device_count())
else:
    print("CUDA is not available. Running on CPU.")

# ------------------------------
# 1. Load the GoEmotions Dataset
# ------------------------------
dataset = load_dataset("go_emotions")
print("\nSample from training set:")
print(dataset["train"][0])  # Show a sample to inspect

# Number of labels
num_labels = dataset["train"].features["labels"].feature.num_classes
print("Number of labels:", num_labels)

# ------------------------------
# 2. Data Preprocessing
# ------------------------------
def convert_labels(label_list):
    """Convert list of label indices to a multi-hot vector of floats."""
    return [1.0 if i in label_list else 0.0 for i in range(num_labels)]

model_checkpoint = "microsoft/deberta-v3-large"  # If memory is an issue, try "microsoft/deberta-v3-base"
tokenizer = DebertaV2Tokenizer.from_pretrained(model_checkpoint)
max_length = 128

def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    tokenized["labels"] = [convert_labels(lbls) for lbls in examples["labels"]]
    return tokenized

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.cast_column("labels", Sequence(Value("float32")))
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ------------------------------
# 3. Model Setup
# ------------------------------
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
)
model.config.problem_type = "multi_label_classification"

if torch.cuda.is_available():
    model = model.to("cuda")

# ------------------------------
# 4. Define Compute Metrics Function
# ------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Binarize predictions using threshold 0.5 by default
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    # Compute subset accuracy (exact match)
    subset_accuracy = np.mean(np.all(predictions == labels, axis=1))
    # Compute micro and macro F1
    micro_f1 = f1_score(labels, predictions, average="micro", zero_division=0)
    macro_f1 = f1_score(labels, predictions, average="macro", zero_division=0)
    return {
        "subset_accuracy": subset_accuracy,
        "micro_f1": micro_f1,
        "macro_f1": macro_f1
    }

# ------------------------------
# 5. Training Arguments & Trainer Setup
# ------------------------------
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # If you have enough GPU RAM, you can increase to 16
    per_device_eval_batch_size=8,
    num_train_epochs=8,             # Increase epochs for better performance
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
)

def custom_data_collator(features):
    batch = DefaultDataCollator()(features)
    batch["labels"] = batch["labels"].float()
    return batch

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=custom_data_collator,
    compute_metrics=compute_metrics,
)

print("\nStarting training...")
trainer.train()

# ------------------------------
# 6. Basic Evaluation with Default Threshold (0.5)
# ------------------------------
print("\nEvaluating on validation set with threshold=0.5...")
val_results = trainer.evaluate()
df_val_results = pd.DataFrame([val_results])
print("Validation Results:")
print(tabulate(df_val_results, headers="keys", tablefmt="fancy_grid"))

print("\nEvaluating on test set with threshold=0.5...")
test_results = trainer.evaluate(eval_dataset=encoded_dataset["test"])
df_test_results = pd.DataFrame([test_results])
print("Test Results:")
print(tabulate(df_test_results, headers="keys", tablefmt="fancy_grid"))

# ------------------------------
# 7. Tune Threshold on Validation Set
# ------------------------------
print("\nFinding best threshold on validation set...")

def find_best_threshold(logits, labels, step=0.05):
    thresholds = np.arange(0.0, 1.0+step, step)
    best_thr = 0.5
    best_f1 = 0.0
    sig_logits = torch.sigmoid(torch.tensor(logits)).numpy()
    for thr in thresholds:
        preds = (sig_logits > thr).astype(int)
        f1 = f1_score(labels, preds, average="micro", zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_thr = thr
    return best_thr, best_f1

val_preds = trainer.predict(encoded_dataset["validation"])
val_logits, val_labels = val_preds.predictions, val_preds.label_ids
best_threshold, best_val_f1 = find_best_threshold(val_logits, val_labels, step=0.01)

print(f"Best threshold on validation set: {best_threshold}")
print(f"Best micro-F1 on validation set with that threshold: {best_val_f1:.4f}")

# ------------------------------
# 8. Evaluate on Test Set Using Best Threshold
# ------------------------------
test_preds = trainer.predict(encoded_dataset["test"])
test_logits, test_labels = test_preds.predictions, test_preds.label_ids
test_probs = torch.sigmoid(torch.tensor(test_logits)).numpy()

test_predictions = (test_probs > best_threshold).astype(int)

# Compute metrics manually
test_subset_accuracy = np.mean(np.all(test_predictions == test_labels, axis=1))
test_micro_f1 = f1_score(test_labels, test_predictions, average="micro", zero_division=0)
test_macro_f1 = f1_score(test_labels, test_predictions, average="macro", zero_division=0)

print("\nTest Results with tuned threshold:")
print(f"  Subset Accuracy: {test_subset_accuracy:.4f}")
print(f"  Micro-F1:        {test_micro_f1:.4f}")
print(f"  Macro-F1:        {test_macro_f1:.4f}")

# ------------------------------
# 9. Save the Model and Tokenizer
# ------------------------------
print("\nSaving the model and tokenizer...")
trainer.save_model("./my_saved_model")  # Saves the best model
tokenizer.save_pretrained("./my_saved_model")
print("Model and tokenizer saved to ./my_saved_model")

# ------------------------------
# 10. Inference Example (Using Best Threshold)
# ------------------------------
def predict_emotions(text, threshold=best_threshold):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits[0]
    probs = torch.sigmoid(logits).cpu().numpy()
    # Use best threshold
    return (probs > threshold).astype(int), probs

sample_text = "I am feeling really happy and excited about today!"
labels_pred, probs_pred = predict_emotions(sample_text)
print("\nSample inference text:", sample_text)
print("Predicted probabilities for each emotion:", probs_pred)
print("Predicted multi-hot labels:", labels_pred)


CUDA is available. Running on GPU.
GPU Name: NVIDIA GeForce RTX 4070 Ti SUPER
GPU Count: 1

Sample from training set:
{'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'id': 'eebbqej'}
Number of labels: 28


c:\Second_Sem\csc392\env_py310\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Second_Sem\csc392\env_py310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saadu\.cache\huggingface\hub\models--microsoft--deberta-v3-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adminis


Starting training...


  0%|          | 100/43416 [00:20<2:25:45,  4.95it/s]

{'loss': 0.2916, 'learning_rate': 1.9953934033536025e-05, 'epoch': 0.02}


...

Test Results:
╒════╤═════════════╤════════════════════════╤═════════════════╤═════════════════╤════════════════╤═══════════════════════════╤═════════════════════════╤═════════╕
│    │   eval_loss │   eval_subset_accuracy │   eval_micro_f1 │   eval_macro_f1 │   eval_runtime │   eval_samples_per_second │   eval_steps_per_second │   epoch │
╞════╪═════════════╪════════════════════════╪═════════════════╪═════════════════╪════════════════╪═══════════════════════════╪═════════════════════════╪═════════╡
│  0 │   0.0816054 │               0.488852 │        0.606616 │         0.45304 │         33.718 │                   160.952 │                  20.138 │       8 │
╘════╧═════════════╧════════════════════════╧═════════════════╧═════════════════╧════════════════╧═══════════════════════════╧═════════════════════════╧═════════╛

Finding best threshold on validation set...


100%|██████████| 679/679 [00:33<00:00, 20.19it/s]


Best threshold on validation set: 0.35000000000000003
Best micro-F1 on validation set with that threshold: 0.6128


100%|██████████| 679/679 [00:33<00:00, 20.11it/s]



Test Results with tuned threshold:
  Subset Accuracy: 0.4514
  Micro-F1:        0.6163
  Macro-F1:        0.4861

Saving the model and tokenizer...
Model and tokenizer saved to ./my_saved_model

Sample inference text: I am feeling really happy and excited about today!
Predicted probabilities for each emotion: [2.0597113e-02 5.7688584e-03 4.0924043e-04 1.1593949e-03 2.3545619e-02
 1.1877004e-02 1.5522424e-03 7.0234756e-03 2.6009441e-03 4.5316090e-04
 1.2884886e-03 2.7227736e-04 2.0298503e-04 7.5846952e-01 3.2438326e-04
 1.5194606e-02 5.8916811e-04 7.1431482e-01 3.5642970e-03 1.1694117e-03
 3.7335148e-03 1.5889428e-03 6.4925775e-03 1.4910299e-02 1.3529226e-04
 6.3893624e-04 5.6049395e-03 7.8487642e-02]
Predicted multi-hot labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0]


In [9]:
from tabulate import tabulate

# Compute metrics manually
test_subset_accuracy = np.mean(np.all(test_predictions == test_labels, axis=1))
test_micro_f1 = f1_score(test_labels, test_predictions, average="micro", zero_division=0)
test_macro_f1 = f1_score(test_labels, test_predictions, average="macro", zero_division=0)

results_dict = {
    "subset_accuracy": [test_subset_accuracy],
    "micro_f1": [test_micro_f1],
    "macro_f1": [test_macro_f1]
}

print("\nTest Results with tuned threshold:")
df_final_results = pd.DataFrame(results_dict)
print(tabulate(df_final_results, headers="keys", tablefmt="fancy_grid"))



Test Results with tuned threshold:
╒════╤═══════════════════╤════════════╤════════════╕
│    │   subset_accuracy │   micro_f1 │   macro_f1 │
╞════╪═══════════════════╪════════════╪════════════╡
│  0 │          0.451446 │   0.616346 │   0.486148 │
╘════╧═══════════════════╧════════════╧════════════╛


In [1]:
# 1. Get the label names from the GoEmotions dataset
label_names = dataset["train"].features["labels"].feature.names

print(label_names)

# 2. A helper function that uses our trained model and tokenizer
#    to predict emotions for a single text prompt
def classify_prompt(text, threshold=best_threshold):
    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Forward pass (inference)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits[0]
    
    # Convert logits to probabilities
    probs = torch.sigmoid(logits).cpu().numpy()
    
    # Apply the best threshold to get binary predictions
    preds = (probs > threshold).astype(int)
    
    # Gather the names of the predicted labels
    predicted_labels = [label_names[i] for i, p in enumerate(preds) if p == 1]
    return predicted_labels, probs

# 3. Test the model on a few custom prompts
test_prompts = [
    "I am feeling really happy and excited about today!",
    "I am disappointed and angry with the results.",
    "This is just okay, nothing special.",
    "I want to order a cheeseburger."
]

for prompt in test_prompts:
    labels, probabilities = classify_prompt(prompt)
    print(f"\nPrompt: {prompt}")
    if labels:
        for lbl in labels:
            idx = label_names.index(lbl)
            print(f"  -> {lbl} (prob={probabilities[idx]:.3f})")
    else:
        print("  -> No emotion predicted above threshold.")


NameError: name 'dataset' is not defined